AMI and ICSI Json Loads

In [1]:
import json
import os

Find json paths and load into list

In [2]:
%ls

README.md                     environment.yml
Untitled.ipynb                environment_op.yml
ami-and-icsi-corpora-master/  solbiati_experiments.ipynb
ami_isci_json_loading.ipynb   test_data_create.ipynb
create_test_data.py           transcripts.pickle
download_data.ipynb           unsupervised_topic_segmentation/


In [10]:
#not using icsi at the moment
#icsi_path='ami-and-icsi-corpora-master/icsi-corpus/output/dialogueActs/

#ami loading
text_path='ami-and-icsi-corpora-master/ami-corpus/output/dialogueActs/'
topic_path='ami-and-icsi-corpora-master/ami-corpus/output/topics/'

text_jsons=[]
topic_jsons=[]

file_paths = os.listdir(topic_path)
file_paths = list(filter(lambda x: x.endswith(".json"),file_paths))

for file_path in file_paths:
    #probably can comment out the text section because text is contained in topic 
    with open( text_path + file_path ) as file:
        json_load = json.loads(file.read())
    text_jsons.append(json_load)
    
    with open( topic_path + file_path ) as file:
        json_load = json.loads(file.read())
    topic_jsons.append(json_load)

len(text_jsons)==len(topic_jsons)


In [25]:
text_jsons[0] #list where text is held in 'text'

print(set([x['label'] for x in text_jsons[0]])) #set of labels
print(set([x['speaker'] for x in text_jsons[0]])) #set of speakers

{'inf', 'el.und', 'el.sug', 'bck', 'stl', 'ass', 'be.pos', 'el.inf', 'oth', 'off', 'und', 'el.ass', 'sug', 'fra'}
{'A', 'B', 'D', 'C'}


Notice that the text has a lot of filler. For example: <vocalsound>, um, uh, mm-hmm, uh-uh, oh 

We can see that the sentence structure is different than our tesimony text. There are many shorter sentences. If we strip these utterances, there will be many empty sentences. Additionally, some utterances are interupted and others contain multiple sentences. 
    

In [23]:
text = [x['text'] for x in text_jsons[0]]
text

Topics 

In [36]:
print(topic_jsons[0][0].keys())
print(set([x['topic'] for x in topic_jsons[0]])) 
print(set([x['description'] for x in topic_jsons[0]])) 


dict_keys(['id', 'topic', 'description', 'dialogueacts', 'subtopics'])
{'discussion', 'marketing expert presentation', 'closing', 'industrial designer presentation', 'interface specialist presentation', 'opening'}
{'None'}
